<a href="https://colab.research.google.com/github/cfretes/ATE_malbran/blob/main/TP2_v01CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


BLOQUE 1: Carga de Librerías y Datos
Objetivo: Importar dependencias y cargar los dos datasets necesarios.


In [2]:
# Librerías estándar para manipulación de datos y cálculos
import pandas as pd
import numpy as np

# Librerías para visualización (EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de Scikit-learn para preprocesamiento y modelado
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Librerías para evaluación de modelos
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Conectar mi drive para la carga de dataframes

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cargar dataset, crear y nombrar DataFrames

In [4]:
# --- Definición de Archivos (Trazabilidad) ---
#Carga de datasets

df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/TP2/smoking_prediction.xlsx - smoking_prediction.csv')
dfe = pd.read_csv ('/content/drive/MyDrive/UOCCB/Diplomatura_IA_UP/TP2/smoking_prediction_entrega.xlsx - smoking_prediction.csv')


In [5]:
df.info()
print(df.head())

dfe.info()
print(dfe.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   50000 non-null  int64  
 1   gender               50000 non-null  object 
 2   age                  50000 non-null  int64  
 3   height(cm)           50000 non-null  int64  
 4   weight(kg)           50000 non-null  int64  
 5   waist(cm)            50000 non-null  float64
 6   eyesight(left)       50000 non-null  float64
 7   eyesight(right)      50000 non-null  float64
 8   hearing(left)        50000 non-null  float64
 9   hearing(right)       50000 non-null  float64
 10  systolic             50000 non-null  float64
 11  relaxation           50000 non-null  float64
 12  fasting blood sugar  50000 non-null  float64
 13  Cholesterol          50000 non-null  float64
 14  triglyceride         50000 non-null  float64
 15  HDL                  50000 non-null 

In [6]:
#Elimino la caracteristica "oral" por ser una constante en ambos Dataframes
df = df.drop('oral', axis=1)
dfe = dfe.drop('oral', axis=1)

Muestreo de DataFrames df/dfe con variable oral eliminada

In [7]:
display(df.head())
display(dfe.head())

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,tartar,smoking
0,0,F,40,155,60,3.38,0.04,0.04,0.04,0.04,...,5.25,0.51,0.04,0.00,0.75,0.79,1.13,0,Y,0
1,1,F,40,160,60,3.38,0.01,0.00,0.04,0.04,...,5.29,0.50,0.04,0.00,0.92,0.79,0.75,0,Y,0
2,2,M,55,170,60,3.33,0.01,0.01,0.04,0.04,...,6.29,0.63,0.04,0.04,0.88,0.67,0.92,0,N,1
3,3,M,40,165,70,3.67,0.05,0.05,0.04,0.04,...,9.42,0.59,0.04,0.04,0.79,1.08,0.75,0,Y,0
4,4,F,40,155,60,3.58,0.04,0.04,0.04,0.04,...,4.46,0.50,0.04,0.00,0.67,0.58,0.92,0,N,0


,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,tartar
0,27358,M,25,160,65,3.42,0.05,0.00,0.04,0.04,...,1.96,3.04,0.63,0.04,0.01,0.75,0.71,0.71,0,Y
1,27364,M,30,180,80,3.46,0.04,0.01,0.04,0.04,...,1.50,4.21,0.59,0.04,0.04,0.79,1.13,1.33,0,N
2,27368,M,55,165,60,3.42,0.00,0.01,0.04,0.04,...,2.13,2.04,0.63,0.04,0.01,1.08,1.29,2.00,1,Y
3,27378,M,20,175,75,3.63,0.05,0.05,0.04,0.04,...,1.71,3.71,0.63,0.04,0.04,0.83,0.58,0.46,0,N
4,27381,M,25,165,80,3.79,0.04,0.04,0.04,0.04,...,2.38,6.63,0.67,0.04,0.04,1.25,1.63,1.96,1,Y


In [8]:
df.describe()

,ID,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,26284.994580,44.189300,164.652000,65.878700,3.415468,0.027999,0.027865,0.041026,0.041048,5.063196,...,2.388346,4.791527,0.594483,0.043708,0.018538,1.094032,1.130193,1.668978,0.213120,0.366580
std,16015.070567,12.090086,9.208778,12.831355,0.386760,0.024137,0.024297,0.006325,0.006389,0.570171,...,0.615541,1.709246,0.066414,0.017509,0.017504,0.828010,1.327440,2.110348,0.409516,0.481875
min,0.000000,20.000000,130.000000,30.000000,2.130000,0.000000,0.000000,0.040000,0.040000,2.960000,...,0.170000,0.040000,0.170000,0.040000,0.000000,0.250000,0.040000,0.040000,0.000000,0.000000
25%,12499.750000,40.000000,160.000000,55.000000,3.170000,0.010000,0.010000,0.040000,0.040000,4.670000,...,1.960000,3.830000,0.550000,0.040000,0.010000,0.790000,0.630000,0.710000,0.000000,0.000000
50%,24999.500000,40.000000,165.000000,65.000000,3.420000,0.040000,0.040000,0.040000,0.040000,5.000000,...,2.290000,4.710000,0.590000,0.040000,0.010000,0.960000,0.880000,1.040000,0.000000,0.000000
75%,40034.250000,55.000000,170.000000,75.000000,3.670000,0.040000,0.040000,0.040000,0.040000,5.420000,...,2.750000,5.670000,0.630000,0.040000,0.040000,1.210000,1.290000,1.830000,0.000000,1.000000
max,55690.000000,85.000000,190.000000,135.000000,5.380000,0.380000,0.380000,0.080000,0.080000,10.000000,...,25.750000,77.500000,0.880000,0.250000,0.460000,54.630000,121.420000,41.630000,1.000000,1.000000


One-Hot Encoding (Conversión a Matriz Dummy)
Objetivo: Transformar las variables categóricas 'gender' y 'tartar' en features binarias (0 y 1).
NOTA: Aplicamos la misma transformación a ambos DataFrames (df y dfe)
para asegurar que tengan el mismo número y nombre de columnas.